In [40]:
#2.1.1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const

df = pd.read_csv("C:/Users/ander/python_decal_fa25/course_assignments/homework9/solar_system.csv")
print("Original shape:", df.shape)

df = df.set_index("Attribute")
df = df.T
df.index.name = "Planet"
df.reset_index(inplace=True)
df.columns.name = None
print("Transposed shape:", df.shape)
print(df.columns)
print(df)

#Questions:
#(a) answered below.
#(b) answered below.
#(c) df.set_index("Attribute") makes the "attribute" the row labels. .T transposes the rows and columns. df.index.name = "Planet" labels the indes as a "Planet". df.reset_index(inplace=True) moves "Planet" out of the index and into a normal column. df.columns.name = None removes the column index name.
#(d) The new shape isnt (11,20) because the csv structure isnt a perfect grid of 11 "Planets" by 20 "Attributes". 
#(e) answerd below.
#(f) Columns with units: ['Mass (10^24kg)', 'Diameter (km)', 'Density (kg/m^3)', 'Gravity (m/s^2)', 'Escape Velocity (km/s)', 'Rotation Period (hours)', 'Length of Day (hours)', 'Distance from Sun (10^6 km)', 'Perihelion (10^6 km)', 'Aphelion (10^6 km)', 'Orbital Period (days)', 'Orbital Velocity (km/s)', 'Orbital Inclination (deg)', 'Obliquity to Orbit (deg)', 'Mean Temperature (C)', 'Surface Pressure (bars)']
#(f) Columns wihtout units: ['Planet', 'Orbital Eccentricity', 'Number of Moons', 'Ring System?', 'Global Magnetic Field?']
#(g) answered below.

print(df.dtypes)
for col in df.columns:
    print(col, df[col].apply(type).unique())
    
#(a) The "Planet" column values are made up of strings
#(b) Storing numeric values as strings are a problem because we aren't able to do math on strings, sorting and comparisons breaks, units and conversions fail, and performance suffers.

for col in df.columns:
    if col not in ["Planet", "Ring System?", "Global Magnetic Field?"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
print(df.dtypes)

#(a) object, int64, and float64
#(b) object is a regular type for non-numeric data. float64 is made up of 64-bit floating point numbers (decimals, scientific notation). int64 is made up of 64-bit integers which are whole numbers.
#(c) "Planet", "Ring System?", and "Global Magnetic Field?" all stay as an object because they arent numeric and shouldnt be converted.

def attach_units(df, old_col, unit, new_col=None):
    df[old_col] = df[old_col].astype(float) * unit
    if new_col:
        df.rename(columns={old_col: new_col}, inplace=True)
    return df
df = attach_units(df, "Mass (10^24kg)", 1e24 * u.kg, "Mass (kg)")
print(df["Mass (kg)"])

#2.3
# semi-major sxis
semi_major = (df["Perihelion (10^6 km)"] + df["Aphelion (10^6 km)"]) / 2
semi_major = semi_major * 1e6
aphelion_index = df.columns.get_loc("Aphelion (10^6 km)")
df.insert(aphelion_index + 1, "Semi-Major Axis (km)", semi_major)
print(df["Semi-Major Axis (km)"])

# orbital period to years
df["Orbital Period (years)"] = (df["Orbital Period (days)"].astype(float) * u.day).apply(lambda x: x.to(u.year).value)
df.drop(columns=["Orbital Period (days)"], inplace=True)
print("Columns after Orbital Period conversion:\n", df.columns)
print(df["Orbital Period (years)"])

earth_period = df.loc[df["Planet"] == "Earth", "Orbital Period (years)"].values[0]
print(f"Earth's orbital period: {earth_period:.4f} years")

# AU conversion
print("1 AU =", const.au)
print("In kilometers:", const.au.to(u.km))

for col in df.columns:
    if "(km)" in col:
        new_col = col.replace("(km)", "(AU)")
        df[new_col] = (df[col].astype(float) * u.km).apply(lambda x: x.to(u.AU).value)

# earth's values
earth_row = df.loc[df["Planet"] == "Earth"]
for col in df.columns:
    if "(AU)" in col:
        print(col, f"{earth_row[col].values[0]:.4f} AU")

# save to the new DataFrame
df.to_csv("units.csv", index=False)
print("DataFrame successfully saved as units.csv")

Original shape: (20, 11)
Transposed shape: (10, 21)
Index(['Planet', 'Mass (10^24kg)', 'Diameter (km)', 'Density (kg/m^3)',
       'Gravity (m/s^2)', 'Escape Velocity (km/s)', 'Rotation Period (hours)',
       'Length of Day (hours)', 'Distance from Sun (10^6 km)',
       'Perihelion (10^6 km)', 'Aphelion (10^6 km)', 'Orbital Period (days)',
       'Orbital Velocity (km/s)', 'Orbital Inclination (deg)',
       'Orbital Eccentricity', 'Obliquity to Orbit (deg)',
       'Mean Temperature (C)', 'Surface Pressure (bars)', 'Number of Moons',
       'Ring System?', 'Global Magnetic Field?'],
      dtype='object')
    Planet Mass (10^24kg) Diameter (km) Density (kg/m^3) Gravity (m/s^2)  \
0  Mercury          0.330          4879             5429             3.7   
1    Venus           4.87         12104             5243             8.9   
2    Earth           5.97         12756             5514             9.8   
3     Moon          0.073          3475             3340             1.6   
4    